In [1]:
# https://stackoverflow.com/questions/36798782/scipy-ckdtree-nearest-neighbor-including-zeros-distance
import random
import numpy as np
import pandas as pd
from scipy.spatial import cKDTree

In [2]:
# Parameter
N = 10

In [3]:
#load
tracks = pd.read_csv('data/tracks.csv', index_col=0, header=[0, 1]);
features= pd.read_csv('data/features.csv', index_col=0, header=[0, 1, 2])
genres=pd.read_csv('data/genres.csv', index_col=0, header=[0])
tracks_raw = pd.read_csv('data/raw_tracks.csv', index_col=0);

In [4]:
# Keep only the genre for each track
label=pd.DataFrame({('label','genre','main'): tracks['track']['genre_top'],
                    ('label','genre','sub'): tracks['track']['genres']})

# Keep on the tracks from small subset
#label = label[tracks['set','subset'] == 'small']


# Keep only the genres we want
#genre_list = ['Rock','Folk','Hip-Hop']#['Hip-Hop', 'Folk',Rock','Jazz','Classical','Pop','Electronic']
#label = label[label['label']['genre']['main'].isin(genre_list)]

#add sub genres
genres_dict=genres['title'].to_dict()
genres_dict={str(key):genres_dict[key] for key in genres_dict}

#label['label']['genre']['sub']=label['label']['genre']['sub'].str.replace('[^0-9| ]','').str.split(' ').apply(lambda x:'|'.join([genres_dict[k] for k in x ]))

dfmax=abs(features).max()
features=features.divide(dfmax)

# Merging
dataSet = features.merge(label, left_index=True, right_on='track_id')
dataSet.head()

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
2             0.000812  0.001826  0.000140  0.000317  0.000465  0.000189   
3             0.000214  0.000266  0.000194  0.000539  0.000519  0.000024   
5             0.000060 -0.000027 -0.000157  0.000161  0.000608  0.000313   
10            0.000419 -0.000102  0.001236 -0.000055  0.000429  0.000355   
20           -0.000022 -0.000069  0.000113  0.000061  0.000243  0.000030   

feature                                               ...       tonnetz  \
statistics                                            ...           std   
number            07        08        09        10    ...            06   
track_id                                              ...                 
2           0.000346  0.000696  0.000712  0.000757    ...      0.104287   
3           0.000319  0.000272  0.000089  0.000349    ...      0.152763   
5          -0.000216 -0.000240  0.000548  0.000586    ...      0.127093   
10          0.000414  0.000415  0.000428  0.001118    ...      0.119880   
20         -0.000068 -0.000211  0.000036 -0.000454    ...      0.183895   

feature          zcr                                                    \
statistics  kurtosis       max      mean    median       min      skew   
number            01        01        01        01        01        01   
track_id                                                                 
2           0.000682  0.459697  0.097525  0.079564  0.000000  0.026764   
3           0.000335  0.466536  0.096328  0.071390  0.000000  0.021985   
5           0.000807  0.375183  0.060493  0.046322  0.000000  0.028089   
10          0.002540  0.452369  0.088283  0.080109  0.000000  0.045365   
20          0.001975  0.469956  0.053785  0.044687  0.003724  0.040851   

feature                 label             
statistics       std    genre             
number            01     main        sub  
track_id                                  
2           0.144817  Hip-Hop       [21]  
3           0.163393  Hip-Hop       [21]  
5           0.105724  Hip-Hop       [21]  
10          0.096156      Pop       [10]  
20          0.073042      NaN  [76, 103]  

[5 rows x 520 columns]

----

#### Loading the custom song features and track id infos:

In [5]:
# Loading custom songs features
features_c = pd.read_csv('data/custom_features.csv', index_col=0, header=[0, 1, 2], sep=',', encoding='utf-8')
tid_c = pd.read_csv('data/custom_tid.csv', index_col=0, header=[0], sep=',', encoding='utf-8')

features_c=features_c.divide(dfmax)
features_c

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
-1            0.000051  0.000198 -0.000422 -0.000120 -0.000117  0.000187   
-2            0.000040 -0.000210 -0.000410 -0.000130  0.000480  0.000845   
-3           -0.000090 -0.000167 -0.000703 -0.000248 -0.000190 -0.000293   
-4            0.000261  0.001273 -0.000526 -0.000121 -0.000072  0.000173   
-5            0.000124 -0.000103 -0.000421 -0.000299  0.000178  0.000118   
-6            0.000048  0.000203 -0.000159 -0.000045 -0.000042 -0.000008   
-7           -0.000024 -0.000149 -0.000495 -0.000229 -0.000114  0.000105   
-8            0.000002  0.000212  0.001317 -0.000029 -0.000211  0.000171   
-9           -0.000056 -0.000198  0.000133  0.000162  0.000067 -0.000125   
-10          -0.000042 -0.000212  0.001567 -0.000020  0.000120  0.000181   
-11           0.000037  0.000220  0.000156  0.000328  0.000158  0.000127   
-12           0.000079 -0.000042  0.000023 -0.000029  0.000028  0.000221   
-13           0.000020 -0.000155 -0.000374 -0.000023  0.000074  0.000099   
-14          -0.000123 -0.000341  0.000787  0.000290 -0.000333 -0.000018   
-15          -0.000048 -0.000036  0.000736  0.000161  0.000075  0.000267   
-16          -0.000098 -0.000280 -0.000051  0.000176 -0.000064  0.000143   
-17          -0.000110 -0.000292 -0.000395 -0.000089 -0.000048 -0.000290   
-18           0.000051  0.000049 -0.000059  0.000341  0.000729  0.000497   
-19           0.000073  0.000385  0.001130  0.000240  0.000445 -0.000052   
-20           0.000085  0.000163  0.000283  0.000172 -0.000076  0.000214   
-21           0.000045 -0.000121  0.000469 -0.000084 -0.000265 -0.000272   
-22           0.000001  0.000340  0.000633  0.000174 -0.000062  0.000008   
-23          -0.000014 -0.000086 -0.000243 -0.000152 -0.000053 -0.000048   

feature                                                   ...      tonnetz  \
statistics                                                ...          std   
number                07        08        09        10    ...           04   
track_id                                                  ...                
-1         -3.528468e-07 -0.000185 -0.000217 -0.000467    ...     0.194425   
-2          1.170729e-04 -0.000020  0.000198 -0.000022    ...     0.283183   
-3         -2.538472e-04  0.000210  0.000169 -0.000311    ...     0.391788   
-4          1.155446e-03 -0.000194 -0.000106 -0.000269    ...     0.482984   
-5          1.176749e-03  0.000554 -0.000880 -0.000513    ...     0.459022   
-6          6.544708e-05 -0.000253 -0.000255 -0.000241    ...     0.254474   
-7         -1.762063e-04  0.000422 -0.000076 -0.000233    ...     0.353697   
-8          2.178658e-04  0.000618 -0.000351  0.000263    ...     0.181892   
-9         -1.565613e-04 -0.000116 -0.000755 -0.000422    ...     0.236728   
-10        -2.484830e-04 -0.000342 -0.001026  0.000864    ...     0.303938   
-11         1.536074e-04 -0.000008 -0.000118  0.000416    ...     0.180341   
-12         3.613668e-04 -0.000041 -0.000216  0.000117    ...     0.240058   
-13        -2.236107e-04 -0.000281 -0.000926 -0.000278    ...     0.282218   
-14        -3.077002e-04 -0.000292 -0.000465 -0.000844    ...     0.340505   
-15         3.156389e-04  0.000100  0.000800  0.000394    ...     0.226666   
-16        -1.156586e-04 -0.000052 -0.000406 -0.000352    ...     0.314990   
-17        -1.890179e-04 -0.000224 -0.000337  0.000685    ...     0.416513   
-18        -2.602179e-05  0.000103  0.000549  0.000121    ...     0.256307   
-19         2.175228e-06 -0.000051 -0.000243 -0.000195    ...     0.177350   
-20         2.593623e-04  0.000257  0.000652 -0.000017    ...     0.200865   
-21         2.114395e-04  0.000181  0.000107 -0.000429    ...     0.350153 

-----
#### Functions

In [6]:
# Dictionnary to link the node position and the track ID
idx_dict={i : dataSet.index[i] for i in range(len(dataSet.index))} # TODO loading/saving is useless actually    
inv_dict = {v: k for k, v in idx_dict.items()}

In [7]:
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, 'Open')+'&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp'+'<a target="_blank" href="{}">{}</a>'.format(val+'/download', 'Download')

In [8]:
def display_result(playlist):
    result=pd.DataFrame({'Artist': tracks.artist.name[playlist], \
                         'Title':tracks.track.title[playlist], \
                         'Links':tracks_raw['track_url'][playlist]}) 

    # Output 
    result.reset_index(inplace=True)
    result.drop(columns='track_id',inplace=True)
    result = result.style.format({'Links':make_clickable})
    
    # Output
    display(result)

----
#### Creating the KD-Tree for the given dataset:

In [9]:
# Convert to np.array
points_ref = np.array([tuple(x) for x in dataSet.drop(columns='label',level=0).values])

# Construct the KD Tree
tree = cKDTree(points_ref)

#### Compute the nearest neighbours in the tree:

In [10]:
#list of the tracks nearest to each custom song
nearest_tracks=dict()

# sub-meta playlist
over_playlist = list()

# For each Playlist we find the nearest point in the KD-Tree with the given songs
for name in tid_c['Playlist name'].unique():
    tid = np.array(tid_c[tid_c['Playlist name'] == name].index.values)
    
    # TODO Tim
    features_c_t = np.array([features_c[features_c.index == idx].values[0] for idx in tid])
    
    # Find the 10th nearest points with Euclidean norm in the tree
    dist, idx = tree.query(features_c_t, k=N, p=2)
    
    # We get the correspond Track ID from the position IDX
    playlist = list([idx_dict[node] for node in idx.ravel()])
    
    # For each song in the playlist we get the best match and store it for later use
    top_tracks = [idx_dict[x] for x in idx[:,0]]
    nearest_tracks.update({name:top_tracks})
    
    # Select in the closest neighbours N random tracks
    playlist = random.sample(playlist,N)
    
    # Add to sub-meta playlist
    over_playlist.append(playlist)

----
#### Printing the top 10 track for each custom songs:

In [11]:
for i,playlist in enumerate(tid_c['Playlist name'].unique()):
    print('Playlist:', playlist)
    display_result(over_playlist[i])

Playlist: Tim


,Artist,Title,Links
0,Guizado,Amplidão,Open Download
1,Gigaboy,I Give Up,Open Download
2,Bernd Burnson,Unter Wasser,Open Download
3,Nick Percev,Flea On Fly,Open Download
4,Tony Gage,DEATHGRIND,Open Download
5,Cosmic Analog Ensemble,An Old Flame,Open Download
6,The Bran Flakes,Countdown To The End (Choo Choo Mix),Open Download
7,M. Takara,Live Sprout Session (03.22.10),Open Download
8,Blah Blah Blah,Going Down to Soho,Open Download
9,Katapulto,Summer Games,Open Download


Playlist: Anael


,Artist,Title,Links
0,Charles Curse,Old Dream,Open Download
1,Derek Clegg,Find Out When We Get There,Open Download
2,Toussaint Morrison,April Father's Day,Open Download
3,Derek Clegg,Easy,Open Download
4,Derek Clegg,Gone,Open Download
5,Derek Clegg,Find Out When We Get There,Open Download
6,Derek Clegg,Human,Open Download
7,Rein,150 Sprint Veloce,Open Download
8,Greg Atkinson,Terracotta Child,Open Download
9,Derek Clegg,Lately,Open Download


Playlist: ZZTop


,Artist,Title,Links
0,Deuxvolt,Dies Irae,Open Download
1,Rotterdam Ska Jazz Foundation,RSJFoundation live at Pacific Parc,Open Download
2,Eaters,The Seeker,Open Download
3,Dev/Null,Rage Template,Open Download
4,Radioactive Sparrow,Please Tell Lisa,Open Download
5,Ervalizer,EXPRAGOT,Open Download
6,Lame Drivers,Color Me Dead (live on WFMU),Open Download
7,Blanket Music,Just Us,Open Download
8,DJMC Haram,Judee On The Sill With Diemindz,Open Download
9,Charlie Wadhams,Arranging My Life,Open Download


Playlist: Lucas


,Artist,Title,Links
0,Those Darlins,Stay On Track / Nitejogger,Open Download
1,Philippe Beer Gabel,Rocketlauncher,Open Download
2,Keshco,Penalty Shot,Open Download
3,Lame Drivers,(as Puppen),Open Download
4,Starring,Iphone,Open Download
5,Winston,La Fin du voyage,Open Download
6,The Rabbits,Live For Today,Open Download
7,Entia,Coordinates,Open Download
8,Upsilon Acrux,So Thereby...and furthermore...thus henceforth...Mono,Open Download
9,Pleasant Grove,Every Heart Is A Meal,Open Download


Playlist: Joacqim


,Artist,Title,Links
0,Tiger Flowers,Cuts,Open Download
1,ONO,Spare,Open Download
2,Jimmy Cousins,Trouble,Open Download
3,Désir Decir,Words,Open Download
4,The Citadel,Útek / Návrat,Open Download
5,The Audacity,Umbrellas,Open Download
6,Croak,Virginia,Open Download
7,The Citadel,Zasiaty Hnev,Open Download
8,The Spunk Lads,Soul Ideal,Open Download
9,Spookey Ruben,Suddenly The Sun,Open Download
